# Setup

## Imports & Constants

May need to change things here if, for instance, default directories for data storage or metadata export column names change.

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import re
import functools
import matplotlib.pyplot as plt
import seaborn as sb
import anndata
import scanpy as sc
import spatialdata
import spatialdata_io as sdio
import numpy as np
import pandas as pd
# import corescpy as cr

ddm = "/mnt/cho_lab" if os.path.exists("/mnt/cho_lab") else "/mnt"  # Spark?
ddl = f"{ddm}/disk2/{os.getlogin()}/data/shared-xenium-library" if (
    "cho" in ddm) else os.path.join(ddu, "shared-xenium-library")
file_mdf = os.path.join(ddl, "samples.csv")  # metadata
path_marker_maps = os.path.join(
    os.path.expanduser("~"), "corescpy/examples/markers_lineages.csv")
col_inflamed, col_stricture = "Inflamed", "Stricture"  # in metadata file
key_uninfl, key_infl, key_stric = "Uninflamed", "Inflamed", "Stricture"
col_sample_id_o = "Sample ID"  # in metadata file
col_condition = "Condition"  # constructed from col_inflamed & col_stricture
col_sample_id, col_condition, col_subject = "Sample", "Condition", "Patient"

## Options & Data

You may make changes here because these are variable options, depending on what you want to analyze.

In [8]:
# Settings/Options
sc.settings.set_figure_params(dpi_save=200, dpi=200)
panel = "TUQ97N"
out_dir = os.path.join(ddl, f"outputs/{panel}/nebraska")
samps = ["Uninflamed-50006B", "Inflamed-50006A", "Stricture-50006C",
         "Uninflamed-50217B", "Inflamed-50217A", "Stricture-50217C",
         "Uninflamed-50336C", "Inflamed-50336B", "Stricture-50336A",
         "Uninflamed-50452A", "Inflamed-50452B", "Stricture-50452C",
         "Uninflamed-50403B", "Inflamed-50403A1", "Inflamed-50403A2",
         "Stricture-50403C1", "Stricture-50403C2"]
samps = ["Uninflamed-50006B", "Inflamed-50006A", "Stricture-50006C",
         "Uninflamed-50217B", "Inflamed-50217A", "Stricture-50217C",
         "Uninflamed-50336C", "Inflamed-50336B", "Stricture-50336A",
         "Uninflamed-50452A", "Inflamed-50452B", "Stricture-50452C",
         "Uninflamed-50403B", "Inflamed-50403A2",
         "Stricture-50403C1", "Stricture-50403C2"]
# samps = ["Uninflamed-50006B", "Inflamed-50006A",
#          "Uninflamed-50217B", "Inflamed-50217A",
#          "Uninflamed-50336C", "Inflamed-50336B",
#          "Uninflamed-50452A", "Inflamed-50452B",
#          "Uninflamed-50403B", "Inflamed-50403A1", "Inflamed-50403A2"]
samps = ["Uninflamed-50006B", "Inflamed-50006A", "Stricture-50006C",
         "Uninflamed-50217B", "Inflamed-50217A", "Stricture-50217C",
         "Uninflamed-50336C", "Inflamed-50336B", "Stricture-50336A",
         "Uninflamed-50452A", "Inflamed-50452B", "Stricture-50452C",
         "Uninflamed-50403B", "Inflamed-50403A2",
         "Stricture-50403C1", "Stricture-50403C2"]
col_cell_type = "bucket_res1pt5_dist0_npc30"
hue_order = ["Uninflamed", "Inflamed", "Stricture"]
palette = ["blue", "red", "yellow"]
layer = "counts"
threshold = 0

# Quantification Files/Options
c_m = "bucket"
c_ann = ["bucket", "bin", "annotation"]
directory = "/home/elizabeth/elizabeth/projects/senescence/analysis"
label = "snc" if "senescence" in directory else "csf"
fff_tx = os.path.join(directory, f"quantification_{label}_tx_cts")
fff_ct = os.path.join(directory, f"quantification_ncells_{label}")

# Gene Categories
assign = pd.read_csv(path_marker_maps).set_index("gene").rename_axis("Gene")
snc_genes = {
    "Cell Cycle Control": ["CDKN2A", "CDKN1A", "PLAUR", "TP53"],
    "SASP": ["IL6", "IL6ST", "IL1A", "CXCL8", "CCL2", "CEBPB",
             "NFKB1", "IGFBP7", "TGFB1"],
    # "DNA Repair": ["ERCC1", "ERCC4"],
    "Fibrosis": ["PLAUR", "IL1", "IL4", "IL6", "IL11", "IL12", "IL13", "IL17",
                 "IL22", "IL23", "IL33", "IL34", "IL36", "TL1A", "TGFβ",
                 "CXCR4", "CCL8", "CCL11", "CHMP1A", "TBX3",
                 "RNF168", "SUCNR1"]
    }

# Data
adatas = {}
for s in samps:
    adatas[s] = sc.read(os.path.join(out_dir, s + ".h5ad"))
    adatas.X = adatas.layers["counts"].copy()

# Constructed Variables
marker_genes_dict = dict(assign["Bucket"].reset_index().groupby(
    "Bucket").apply(lambda x: list(pd.unique(list(set(
        x.Gene).intersection(adatas[s].var_names))))))  # cell type markers
snc_genes_dict = dict(pd.Series(snc_genes).apply(
    lambda x: list(set(x).intersection(adatas[s].var_names))))
hue = dict(hue=col_condition, hue_order=hue_order, palette=palette)
c_l = "leiden_" + "_".join(col_cell_type.split("_")[1:])  # Leiden version

# Annotations
fmrs = {}
for x in samps:
    fann = os.path.join(out_dir, "annotation_dictionaries", str(
        f"{x}___{c_l}_dictionary.xlsx"))
    if os.path.exists(fann):
        fmr = pd.read_excel(fann).astype(str).replace("nan", np.nan)
        fmrs[x] = fmr.set_index(fmr.columns[0]).rename_axis(c_l)
fmrs = pd.concat(fmrs, names=["Sample", c_l])

/tmp/ipykernel_65396/48061552.py:50: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  "Bucket").apply(lambda x: list(pd.unique(list(set(
/tmp/ipykernel_65396/48061552.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  "Bucket").apply(lambda x: list(pd.unique(list(set(
/tmp/ipykernel_65396/48061552.py:63: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fmr = pd.read_exc

## Samples

In [12]:
# Load Metadata
m_d = (pd.read_excel if file_mdf[-4:] == "xlsx" else pd.read_csv)(
    file_mdf, dtype={"Slide ID": str}).rename({
        "Name": col_subject, "Inflammation": col_inflamed}, axis=1)
m_d.loc[:, col_condition] = m_d.apply(lambda x: "Stricture" if x[
    col_stricture].lower() in ["stricture", "yes"] else x[
        col_inflamed].capitalize(), axis=1)  # inflamation/stricture condition
m_d.loc[:, col_sample_id] = m_d[[col_condition, col_sample_id_o]].apply(
    "-".join, axis=1)
m_d = m_d.set_index(col_sample_id).loc[samps]
print(m_d[[col_subject, col_condition]].reset_index(0)[
    col_condition].value_counts())
m_d.reset_index().set_index([col_subject, col_condition]).sort_index()[
    ["Age", "Sex", "Race", "Hispanic", "Diagnosis", "Disease_Status"]]

# Add Metadata & Annotations to AnnData Objects & Concatenate
for s in adatas:
    adatas[s].obs.loc[:, col_condition] = m_d.loc[s][col_condition]
    adatas[s].obs.loc[:, col_subject] = m_d.loc[s][col_subject]
    adatas[s].obs.loc[:, col_cell_type] = adatas[s].obs[c_l].astype(
        str).replace(dict(fmrs.loc[s][c_m]))  # annotations
adata = anndata.AnnData.concatenate(
    *[adatas[x] for x in adatas], join="outer", batch_key=col_sample_id,
    batch_categories=list(adatas.keys()), index_unique="-", fill_value=None,
    uns_merge="same")  # concatenate adata

Condition
Inflamed      6
Uninflamed    5
Name: count, dtype: int64


/tmp/ipykernel_65396/1800670938.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '50403' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  adatas[s].obs.loc[:, col_subject] = m_d.loc[s][col_subject]
/tmp/ipykernel_65396/1800670938.py:23: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = anndata.AnnData.concatenate(


# UMAPs

In [ ]:
fig, axes = plt.subplots(len(m_d[col_subject].unique()),
                         len(m_d[col_condition].unique()),
                         dpi=200, figsize=(60, 60))
for r, s in enumerate(m_d[col_subject].unique()):
    for c, g in enumerate(m_d[col_condition].unique()):
        lib = m_d.reset_index().set_index([col_subject, col_condition]).loc[
            s].loc[g][col_sample_id]
        sc.pl.umap(adatas[lib], color=col_cell_type, title="",
                   frameon=False, show=False, ax=axes[r, c],
                   legend_loc="on data")
        axes.flatten()[i].set_title(x)
        axes[r, c].set_title(f"{lib} (Age {m_d['Age'].loc[lib]})")

# Spatial Plots

In [ ]:
for s in selves:
    m_d = metadata.loc[s._library_id]
    fig = s.plot_spatial(color="bucket_res1pt5_dist0_npc30")
    fig.set_title(f"{s._library_id} (Age {m_d['Age']})")
    fig.fig.set_dpi(200)

# Quantifications

## Counts

In [ ]:
thresh = [1, 2, 3, 4]
genes = ["CSF1", "CSF2", "CSF3", "IL7R", "KLRB1", "RORC"]

directory = "/home/elizabeth/elizabeth/projects/csf2rb/analysis"
fff = os.path.join(directory, f"quantification_ncells_csf.xlsx")

cts = {}
for v in adatas:
    cts[v] = pd.DataFrame(
        dict(Gene=genes * len(thresh), Threshold=[
            t for t in thresh for g in genes], Count=np.nan,
             Total=[adatas[v].n_obs] * len(genes) * len(thresh))
        ).set_index(["Threshold", "Gene"])
    for t in thresh:  # iterate GEX thresholds
        cts[v].loc[t, "Count"] = [np.sum(adatas[v][:, g].X >= t)
                                  for g in cts[v].loc[t].index]  # sum GEX
cts = pd.concat(cts, names=["Sample"])
cts.to_excel(fff)

In [211]:
genes = ["CSF1", "CSF2", "CSF3", "IL7R", "KLRB1", "RORC"]
genes_markers = ["IL7R", "KLRB1", "RORC"]

i_x = pd.Index(adatas.keys(), name=col_sample_id)
cell_type_label = "/".join([f"{g}+" for g in genes_markers])
col_label = "_".join(genes_markers)
for v in adatas:
    for g in genes:
        thresh_dummy = adatas[v][:, g].X >= 1
        if "toarray" in dir(thresh_dummy):
            thresh_dummy = thresh_dummy.toarray()
        adatas[v].obs.loc[:, g] = [g + "+" if i[0] else g + "-"
                                   for i in thresh_dummy]
    adatas[v].obs.loc[:, col_label] = adatas[v].obs.loc[
        :, genes_markers].apply(lambda x: "/".join(x), axis=1)
n_cells_in_cluster = pd.Series([np.sum(
  adatas[v].obs[col_label] == cell_type_label) for v in adatas], index=i_x)
n_genepos = pd.concat([
    pd.Series([adatas[v].var.loc[g]["n_cells"] for v in adatas], index=i_x)
    for g in genes], keys=genes, names=["Gene"])
qqs = pd.Series([adatas[v].n_obs for v in adatas], index=i_x).to_frame(
    "N Total Cells").join(n_cells_in_cluster.to_frame(
        f"N {cell_type_label}").join(n_genepos.to_frame("N Gene Positive")))

n_cells_pos_in_cluster = pd.concat([
    pd.Series([np.sum(adatas[v][adatas[v].obs[col_label] == cell_type_label][
        :, g].X >= 1) for v in adatas], index=i_x)
    for g in genes], keys=genes, names=["Gene"])
percent_cells_pos_in_cluster = pd.concat([
    pd.Series([np.mean(adatas[v][adatas[v].obs[col_label] == cell_type_label][
        :, g].X >= 1) * 100 for v in adatas], index=i_x)
    for g in genes], keys=genes, names=["Gene"])
qqq = n_cells_pos_in_cluster.to_frame("N").join(
        percent_cells_pos_in_cluster.to_frame(f"Percent"))


among_geneposcells_n_in_cluster = pd.concat([
    pd.Series([np.sum(adatas[v][adatas[v].obs[g] == f"{g}+"].obs[
        col_label] == cell_type_label) for v in adatas], index=i_x)
    for g in genes], keys=genes, names=["Gene"])
among_geneposcells_perc_in_cluster = pd.concat([
    pd.Series([np.mean(adatas[v][adatas[v].obs[g] == f"{g}+"].obs[
        col_label] == cell_type_label) * 100 for v in adatas], index=i_x)
    for g in genes], keys=genes, names=["Gene"])
qqe = among_geneposcells_n_in_cluster.to_frame("N").join(
    among_geneposcells_perc_in_cluster.to_frame("Percent")).drop(
        genes_markers, level=0)

qqx = pd.concat([qqs, qqq, qqe], keys=[
    "Sample", f"{cell_type_label}", f"Gene+"], names=["Reference"], axis=1)
qqx

Reference                     Sample                                       \
                       N Total Cells N IL7R+/KLRB1+/RORC+ N Gene Positive   
Gene Sample                                                                 
CSF1 Uninflamed-50006B         78976                  218           12051   
     Inflamed-50006A          220569                  779           38374   
     Uninflamed-50217B        773732                 4172          128497   
     Inflamed-50217A          972278                 2820          142566   
     Uninflamed-50336C        378219                 3324           61706   
...                              ...                  ...             ...   
RORC Uninflamed-50452A        312629                 2165           25076   
     Inflamed-50452B          257679                  438            4850   
     Uninflamed-50403B        648692                 1385            9361   
     Inflamed-50403A1         178246                 1400           10362   
     Inflamed-50403A2         202062                 1476           11278   

Reference              IL7R+/KLRB1+/RORC+              Gene+            
                                        N     Percent      N   Percent  
Gene Sample                                                             
CSF1 Uninflamed-50006B                 57   26.146789   57.0  0.472990  
     Inflamed-50006A                  120   15.404365  120.0  0.312712  
     Uninflamed-50217B                832   19.942474  832.0  0.647486  
     Inflamed-50217A                  612   21.702128  612.0  0.429275  
     Uninflamed-50336C                606   18.231047  606.0  0.982076  
...                                   ...         ...    ...       ...  
RORC Uninflamed-50452A               2165  100.000000    NaN       NaN  
     Inflamed-50452B                  438  100.000000    NaN       NaN  
     Uninflamed-50403B               1385  100.000000    NaN       NaN  
     Inflamed-50403A1                1400  100.000000    NaN       NaN  
     Inflamed-50403A2                1476  100.000000    NaN       NaN  

[66 rows x 7 columns]

In [ ]:
qqx.to_excel(os.path.join(directory, "_".join(genes_markers) + ".xlsx"))

## Native Plotting

### Facet=Cell Type; Rows=Condition

In [ ]:
for j, k in enumerate(["var", "group"]):
    fig, axes = plt.subplots(*cr.pl.square_grid(len(in_all_cond)),
                             figsize=(40, 20))
    for i, x in enumerate(in_all_cond):
        sc.pl.dotplot(
            adata[adata.obs[col_cell_type] == x], snc_genes_dict, layer=layer,
            groupby=col_condition, title=x, ax=axes.flatten()[i],
            use_raw=False, show=False, expression_cutoff=threshold,
            figsize=(40, 20), standard_scale=k)
        # d_p.add_totals().style(dot_edge_color="black", dot_edge_lw=0.5).show()
        # axes.flatten()[i].set_title(x)
    fig.suptitle(f"GEX (Normalized by {['Gene', 'Condition'][j]})")
    plt.subplots_adjust(hspace=1, wspace=1, top=0.85)
    fig.show()

### Facet=Condition; Rows=Cell Type

In [ ]:
for j, k in enumerate(["var", "group"]):
    fig, axes = plt.subplots(*cr.pl.square_grid(
        len(adata.obs[col_condition].unique())), figsize=(40, 20))
    for i, x in enumerate(adata.obs[col_condition].unique()):
        sc.pl.dotplot(
            adata[adata.obs[col_condition] == x], snc_genes_dict, layer=layer,
            groupby=col_cell_type, ax=axes.flatten()[i],
            use_raw=False, show=False, expression_cutoff=threshold,
            # title=x,
            figsize=(40, 20), standard_scale=k)
        # d_p.add_totals().style(dot_edge_color="black", dot_edge_lw=0.5).show()
        axes.flatten()[i].set_title(x)
    fig.suptitle(f"GEX (Normalized by {['Gene', 'Cluster'][j]})")
    plt.subplots_adjust(hspace=1, wspace=0.5, top=0.85)
    fig.show()

### By Sample

In [ ]:
for j, k in enumerate(["var", "group"]):
    fig, axes = plt.subplots(len(m_d[col_subject].unique()),
                             len(m_d[col_condition].unique()),
                             dpi=200, figsize=(60, 60))
    for r, s in enumerate(m_d[col_subject].unique()):
        for c, g in enumerate(m_d[col_condition].unique()):
            lib = m_d.reset_index().set_index([
                col_subject, col_condition]).loc[s].loc[g][col_sample_id]
            sc.pl.dotplot(
                adatas[lib], snc_genes_dict, layer=layer,
                groupby=col_cell_type, ax=axes[r, c],
                use_raw=False, show=False, expression_cutoff=threshold,
                figsize=(40, 20), standard_scale=k)
            axes[r, c].set_title(f"{lib} (Age {m_d['Age'].loc[lib]})")
    fig.suptitle(f"GEX (Normalized by {['Gene', 'Cluster'][j]})")
    plt.subplots_adjust(hspace=1, wspace=0.5, top=0.85)
    fig.show()

In [28]:
pd.concat([adatas[v].obs[["cell_filter_ncounts", "cell_filter_ngene"]].iloc[
    0].apply(lambda x: x.split(", ")[0][1:]) for v in adatas],
          keys=adatas.keys()).unstack(1).sort_values("cell_filter_ncounts")

,cell_filter_ncounts,cell_filter_ngene
Uninflamed-50217B,15,3
Inflamed-50217A,15,3
Uninflamed-50336C,15,3
Inflamed-50336B,15,3
Uninflamed-50403B,15,3
Inflamed-50403A1,15,3
Inflamed-50403A2,15,3
Uninflamed-50006B,50,10
Inflamed-50006A,50,10
Uninflamed-50452A,50,10


In [ ]:
# layer = "counts"
# threshold = 0

# in_all_cond = [i for i in adata.obs[col_cell_type].unique() if all(
#     (any(adata[adata.obs[col_cell_type] == i].obs[col_condition] == x)
#      for x in hue_order))]  # clusters present in all condition-samples
# cts = {}
# for c in snc_genes:
#     cts = {}
#     for g in snc_genes[c]:
#         cts[g] = pd.concat([pd.Series([100 * np.sum(adatas[a][adatas[a].obs[
#             col_cell_type] == x][:, g].X >= threshold) / adatas[a][adatas[
#                 a].obs[col_cell_type] == x].obs.shape[0] for x in in_all_cond],
#                                       index=all_cond) for a in adatas],
#                            keys=adatas.keys(), names=["Sample"])
#     percs_txs[c] = pd.concat(stx, names=["Gene"])
# percs_txs = pd.concat(percs_txs, names=["Category"])

## Saved Quantifications

### Load

In [ ]:
# Transcript Ratios
txs = pd.read_csv(f"{fff_tx}.csv")
tx_cl = pd.read_csv(fff_tx + "_by_cluster.csv", index_col=np.arange(
    len(txs.index.names))).reset_index().astype(
        {c_l: str}).set_index(tx_cl.index.names)
tx_cl = tx_cl.reset_index().astype({c_l: str}).set_index(
    tx_cl.index.names).drop(set(c_ann).intersection(
        tx_cl.columns), axis=1).join(fmrs[c_ann], on=["Sample", c_l])
tx_cl = tx_cl.reset_index().drop_duplicates().astype(
    {c_l: int}).set_index(tx_cl.index.names).sort_index().reset_index(
        ).astype({c_l: "string"}).set_index(tx_cl.index.names)

# Gene+ Cell Ratios
cts = pd.read_csv(fff_ct + ".csv", index_col=0)
cts_cl = pd.read_csv(fff_ct + "_by_cluster.csv", header=[0, 1],
                     index_col=[0, 1, 2])
cts_cl_s = pd.concat([fmrs[c_ann].reset_index().astype("string").set_index(
    fmrs.index.names)], axis=1, keys=["Annotation"])
cts_cl_s = cts_cl_s.reset_index().drop_duplicates().astype(str).set_index(
    cts_cl_s.index.names)
cts_cl = cts_cl[cts_cl.columns.difference(cts_cl_s.columns)].reset_index(
    ).drop_duplicates().astype(str).set_index(
        cts_cl.index.names).join(cts_cl_s)

### Plot

#### Transcripts

In [ ]:
y_label = "Gene Transcript Counts: Percent of Total Counts"
dff = tx_cl.set_index(c_ann, append=True).apply(lambda x: tx_cl[
    x.name] / tx_cl["total_counts"]).drop(
        "total_counts", axis=1).rename_axis("Gene", axis=1).stack()
dff = dff * 100
dff = dff.to_frame(y_label).join(
    metadata_o[[col_sample_id, col_subject, col_condition]].set_index(
        col_sample_id), how="left").join(fmrs[c_ann]).reset_index(
            ).drop_duplicates()
if dff[y_label].max() > 1:
    raise ValueError(f"Percentages > 1: {dff[dff[y_label] > 1]}")
for x in [None, col_subject]:  # aggregate across subjects, or facet
    fig = sb.catplot(dff.dropna(subset=[c_m]), x=c_m, y=y_label,
                     kind="box" if x is None else "bar",
                     sharex=False, sharey=False, row=x, col="Gene",
                     margin_titles=True, **hue)
    fig.set_xticklabels(rotation=90)
    fig.set_titles(col_template="{col_name}", row_template="{row_name}")
    fig.fig.set_size_inches(60, 25)
    plt.subplots_adjust(hspace=0.8)
    fig.fig.tight_layout()
    plt.show()

#### Cells

In [ ]:
vvr = cts_cl.copy()["Percent"].rename_axis("Label", axis=1).dropna(
    how="all").stack().to_frame("Percent").astype(float).join(fmrs[[c_m]])
vvr = vvr.join(vvr.groupby("Sample").apply(lambda x: x.name.split("-")[
    0]).to_frame(col_condition)).reset_index().drop_duplicates()
vvr = vvr[vvr.apply(lambda x: "|" not in x[c_m], axis=1)]  # no mixes
for x in vvr.Threshold.unique():
    fig = sb.catplot(vvr[vvr.Threshold == x], x=c_m, y="Percent",
                     sharey=False, kind="box", margin_titles=True,
                     col="Label", col_wrap=2, sharex=False, **hue)
    fig.set_titles(col_template="{col_name}", row_template="{row_name}")
    fig.set_xticklabels(rotation=90)
    fig.fig.set_size_inches(35, 25)
    fig.fig.suptitle(f"Threshold = {x}")
    fig.fig.set_dpi(100)
    fig.fig.tight_layout()

# More Analysis

In [ ]:
for s in selves:
    s.run_composition_analysis(col_condition=col_condition)